In [1]:
import matplotlib.pyplot as plt

import datetime
import pandas as pd
import numpy as np
import numpy.random as rd
import scipy.stats
import yfinance as yf
import math
import time

import numba
from numba import jit
# from numba.types import string
# from numba import int32, float32 
# from numba.experimental import jitclass

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
def get_data(index_list):
    df_final = pd.DataFrame()

    for acao in index_list:
        try:
            dados = yf.Ticker(acao).history(period="1mo",auto_adjust=True).reset_index()  
            dados["Nome"] = acao

            df_final = pd.concat([df_final,dados],axis=0)


        except:
            print ("Erro")

    return df_final

### Sem Numba

In [10]:
class yahoo_data_sem_numba:
    def __init__(self, df_stocks, janela_1, janela_2, fator_decaimento_ewma):
        self.df_final = df_stocks
        self.janela_1 = janela_1
        self.janela_2 = janela_2
        self.fator_decaimento_ewma = fator_decaimento_ewma
        
        self.get_log_return()
        self.get_vol_ewma()
        self.eng_vars()
        
        self.df_final = self.df_final[["Date", "Close", "Nome", "log_ret", "vol_ewma", "Janela_1", "Janela_2", "Indicador"]]
            
            
    def get_log_return(self):
        self.df_final["log_ret"] = 0
        self.df_final.dropna(subset=["Close"], inplace=True)
        
        self.aux = pd.DataFrame()
        
        for acao in self.df_final["Nome"].unique():
            filtrado = self.df_final[self.df_final['Nome']==acao]
            filtrado['log_ret'] = np.log(filtrado["Close"]) - np.log(filtrado["Close"].shift(1))            
            
            self.aux = pd.concat([self.aux,filtrado],axis=0)
            
        self.df_final = self.aux
        
    def get_vol_ewma(self):
        """
        Usa a série de Retornos e a constante para calcular a Volatilidade EWMA.
        """
        
        Lambda = self.fator_decaimento_ewma
        self.aux = pd.DataFrame()
        
        for acao in self.df_final["Nome"].unique():
            filtrado = self.df_final[self.df_final['Nome']==acao]
            ReturnSeries = filtrado["log_ret"]
            
            SampleSize = len(ReturnSeries)
            Average = ReturnSeries.mean()

            e = np.arange(SampleSize-1,-1,-1)
            r = np.repeat(Lambda,SampleSize)
            vecLambda = np.power(r,e)

            sxxewm = (np.power(ReturnSeries-Average,2)*vecLambda).sum()
            Vart = sxxewm/vecLambda.sum()
            EWMAVol = np.sqrt(Vart)     
            
            filtrado["vol_ewma"] = EWMAVol
            
            
            self.aux = pd.concat([self.aux,filtrado],axis=0)
            
            
        self.df_final = self.aux  
        
        
    def eng_vars(self):
        self.df_final["Volume"] = self.df_final["Volume"].astype(np.float64)
        self.aux = pd.DataFrame()  
        
        def get_movel(x, days):
            if (x.dtype == float):
                return x.rolling(days).mean()
            else:
                return x     
         
        for acao in self.df_final["Nome"].unique():
            filtrado = self.df_final[self.df_final['Nome']==acao]        
            
            filtrado['Janela_1'] = filtrado[["Close"]].apply(lambda x: get_movel(x, self.janela_1),axis=0)
            filtrado['Janela_2'] = filtrado[["Close"]].apply(lambda x: get_movel(x, self.janela_2),axis=0)

            self.aux = pd.concat([self.aux,filtrado],axis=0)
            
        self.df_final = self.aux
        self.df_final = self.df_final.fillna(0)
        self.df_final["Indicador"] = self.df_final.apply(lambda x: 1 if x["Janela_1"] > x["Janela_2"] else 0, axis=1)

### Com Numba

In [11]:
@jit
def _get_log_return(df_final, unicos):
    splited = np.split(df_final, unicos)
    aux = []

    for acao in splited:
        aux = np.concatenate((aux, np.diff(np.log(acao))), axis=None)

    return aux

@jit
def _get_vol_ewma(ret, unicos, fator_decaimento_ewma):
    Lambda = fator_decaimento_ewma
    splited = np.split(ret, unicos)
    aux = []

    for acao in splited:
        ReturnSeries = acao

        SampleSize = len(ReturnSeries)
        Average = ReturnSeries.mean()

        e = np.arange(SampleSize-1,-1,-1)
        r = np.repeat(Lambda,SampleSize)
        vecLambda = np.power(r,e)

        sxxewm = (np.power(ReturnSeries-Average,2)*vecLambda).sum()
        Vart = sxxewm/vecLambda.sum()
        EWMAVol = math.sqrt(Vart)
        
        EWMAVol = np.repeat(EWMAVol,SampleSize+1)

        aux = np.concatenate((aux, EWMAVol), axis=None)


    return aux

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


@jit
def _eng_vars(df_final, unicos, janela1, janela2):
    splited = np.split(dados["Close"].values, unicos)
    aux_1 = []
    aux_2 = []


    for acao in splited:
        janela_1 = moving_average(acao, janela1)
        janela_2 = moving_average(acao, janela2)
        
        val = len(acao) - len(janela_1)
        for i in range(val):
            janela_1 = np.insert(janela_1, i, 0)

        val = len(acao) - len(janela_2)
        for i in range(val):
            janela_2 = np.insert(janela_2, i, 0)   

        aux_1 = np.concatenate((aux_1, janela_1), axis=None)
        aux_2 = np.concatenate((aux_2, janela_2), axis=None)

    indicador = []  

            
    for i in range(len(aux_1)):
        if aux_1[i] > aux_2[i]:
            indicador.append(1)
        else:
            indicador.append(0)
            
    return indicador, aux_1, aux_2

class yahoo_data_com_numba:
    def __init__(self, df_stocks, unicos, janela_1, janela_2, fator_decaimento_ewma):
        
        self.df_final = df_stocks
        self.unicos = unicos
        self.janela_1 = janela_1
        self.janela_2 = janela_2
        self.fator_decaimento_ewma = fator_decaimento_ewma
        
        self.get_log_return()
        self.get_vol_ewma()
        self.eng_vars()
        
        self.result = np.matrix([self.df_final, self.vol_ewma, self.indicador])
        
    def get_log_return(self):
        self.ret = _get_log_return(self.df_final, self.unicos)

    def get_vol_ewma(self):
        self.vol_ewma = _get_vol_ewma(self.ret, self.unicos, self.fator_decaimento_ewma)        
        
    def eng_vars(self):
        self.indicador, self.janela_1, self.janela_2 = _eng_vars(self.df_final, self.unicos, self.janela_1, self.janela_2)


In [20]:
index_list = ['ABEV3.SA','B3SA3.SA','VALE3.SA']
dados = get_data(index_list)
unicos  = dados["Nome"].nunique()

In [21]:
inicio = time.time()
operacao1 = yahoo_data_sem_numba(dados, 7, 10, 0.94)
resultado1 = operacao1.df_final
fim = time.time()
tempo_sem_numba = fim-inicio


inicio = time.time()
operacao2 = yahoo_data_com_numba(dados["Close"].values, unicos, 7, 10, 0.94)
resultado2 = operacao2.result
fim = time.time()

tempo_com_numba = fim-inicio

melhoria = (tempo_sem_numba - tempo_com_numba)/(tempo_sem_numba)*100

In [22]:
print(f"Tempo de processamento do método sem utilizar Numba: {tempo_sem_numba} \nTempo de processamento do método com Numba: {tempo_com_numba} \nMelhoria de: {melhoria}%")

Tempo de processamento do método sem utilizar Numba: 0.04403567314147949 
Tempo de processamento do método com Numba: 0.0009999275207519531 
Melhoria de: 97.72927844763643%
